In [186]:
#load the library
import numpy as np
import pandas as pd

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.datasets import load_iris 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

import lightgbm as lgb
import optuna
import warnings
from tqdm import tqdm
import time
from sklearn.decomposition import TruncatedSVD, PCA
import joblib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [187]:
#import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('gender_submission.csv')

In [188]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [189]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [190]:
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [191]:
train.shape

(891, 12)

In [192]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Fill NAs

In [193]:
sub.PassengerId.to_list() == test.PassengerId.to_list()

True

In [194]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [195]:
## fill age
# train.Age = train.Age.fillna(np.mean(train.Age))
train.Age = train.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))

In [196]:
train.Cabin = train.Cabin.fillna('Unknown')

In [197]:
train.Embarked = train.Embarked.fillna('Unknown')

In [198]:
train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [199]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [200]:
# test.Age = test.Age.fillna(np.median(train.Age))
test.Age = test.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
test.Fare = test.Fare.fillna(np.mean(test.Fare))

In [201]:
test.Cabin = test.Cabin.fillna('Unknown')

In [202]:
test.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

## Split folds

In [203]:
#do k fold validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [204]:
train['fold'] = -999
for fold_id, (train_index, valid_index) in enumerate(skf.split(X=train, y=train.Survived.values)):
    train.loc[valid_index, 'fold'] = fold_id

In [205]:
train.fold.value_counts()

0    179
1    178
2    178
3    178
4    178
Name: fold, dtype: int64

## Feature Engineering

#### 1. Name Title (Change the name to title and label encoding)

In [206]:
train['Title'] = train.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [207]:
train['Title'] = train['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 
                                             'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [208]:
test['Title'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(test['Title'], test['Sex'])

Sex,female,male
Title,,
Col,0,2
Dona,1,0
Dr,0,1
Master,0,21
Miss,78,0
Mr,0,240
Mrs,72,0
Ms,1,0
Rev,0,2


In [209]:
test['Title'] = test['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 
                                             'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

test['Title'] = test['Title'].replace('Mlle', 'Miss')
test['Title'] = test['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace('Mme', 'Mrs')

In [210]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

train['Title'] = train['Title'].map(title_mapping)
train['Title'] = train['Title'].fillna(0)
test['Title'] = test['Title'].map(title_mapping)
test['Title'] = test['Title'].fillna(0)

In [211]:
# train = train.drop('Name', axis=1)
# test = test.drop('Name', axis=1)

In [212]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,4,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,3,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,0,1


In [213]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Unknown,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,Unknown,S,3
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Unknown,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,Unknown,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,Unknown,S,3


### 2.  Cut the Age and Fare column to different bins 

In [214]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,fold,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.112424,0.523008,0.381594,32.204208,1.997755,1.728395
std,257.353842,0.486592,0.836071,13.304424,1.102743,0.806057,49.693429,1.415800,1.030039
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,21.500000,0.000000,0.000000,7.910400,1.000000,1.000000
50%,446.000000,0.000000,3.000000,26.000000,0.000000,0.000000,14.454200,2.000000,1.000000
75%,668.500000,1.000000,3.000000,36.000000,1.000000,0.000000,31.000000,3.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,4.000000,5.000000


In [215]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Title
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,29.297847,0.447368,0.392344,35.627188,1.741627
std,120.810458,0.841838,13.038856,0.896760,0.981429,55.840500,1.006091
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,996.250000,1.000000,22.000000,0.000000,0.000000,7.895800,1.000000
50%,1100.500000,3.000000,25.000000,0.000000,0.000000,14.454200,1.000000
75%,1204.750000,3.000000,36.375000,1.000000,0.000000,31.500000,2.000000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200,5.000000


In [216]:
train['Fare'] = pd.qcut(train['Fare'], 10)
test['Fare'] = pd.qcut(test['Fare'], 10)

In [217]:
train['Fare'].value_counts()

(7.854, 8.05]        106
(-0.001, 7.55]        92
(27.0, 39.688]        91
(21.679, 27.0]        89
(39.688, 77.958]      89
(14.454, 21.679]      88
(7.55, 7.854]         87
(77.958, 512.329]     87
(10.5, 14.454]        84
(8.05, 10.5]          78
Name: Fare, dtype: int64

In [218]:
train['Age'].value_counts()

25.00    117
21.50     42
40.00     34
30.00     34
24.00     30
        ... 
80.00      1
0.42       1
34.50      1
66.00      1
12.00      1
Name: Age, Length: 89, dtype: int64

In [219]:
train['Age'] = pd.qcut(train['Age'], 10)
test['Age'] = pd.qcut(test['Age'], 10)

In [220]:
train['Age'].value_counts()

(22.0, 25.0]     164
(34.0, 40.0]     116
(26.0, 30.0]     101
(0.419, 16.0]    100
(20.0, 22.0]      94
(47.0, 80.0]      89
(16.0, 20.0]      79
(30.0, 34.0]      69
(40.0, 47.0]      61
(25.0, 26.0]      18
Name: Age, dtype: int64

### 3. split sibsp column into different case based on the size

In [221]:
train['SibSp'].value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [222]:
test['SibSp'].value_counts()

0    283
1    110
2     14
3      4
4      4
8      2
5      1
Name: SibSp, dtype: int64

In [223]:
#0 -- small family size; 1 -- middle family size; 2 -- large family size
train['SibSp'].loc[train['SibSp'] < 3] = 0
train['SibSp'].loc[(train['SibSp'] < 5) & (train['SibSp'] >= 3)] = 1
train['SibSp'].loc[train['SibSp'] >= 5] = 2

test['SibSp'].loc[test['SibSp'] < 3] = 0
test['SibSp'].loc[(test['SibSp'] < 5) & (test['SibSp'] >= 3)] = 1
test['SibSp'].loc[test['SibSp'] >= 5] = 2

In [224]:
train['SibSp'].value_counts()

0    845
1     34
2     12
Name: SibSp, dtype: int64

In [225]:
test['SibSp'].value_counts()

0    407
1      8
2      3
Name: SibSp, dtype: int64

In [226]:
# train['Fare'] = str(train['Fare'])
# test['Fare'] = str(test['Fare'])

# train['Age'] = str(train['Age'])
# test['Age'] = str(test['Age'])

# train['SibSp'] = str(train['SibSp'])
# test['SibSp'] = str(test['SibSp'])

In [227]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,"(20.0, 22.0]",0,0,A/5 21171,"(-0.001, 7.55]",Unknown,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,"(34.0, 40.0]",0,0,PC 17599,"(39.688, 77.958]",C85,C,4,3
2,3,1,3,"Heikkinen, Miss. Laina",female,"(25.0, 26.0]",0,0,STON/O2. 3101282,"(7.854, 8.05]",Unknown,S,3,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,"(34.0, 40.0]",0,0,113803,"(39.688, 77.958]",C123,S,3,3
4,5,0,3,"Allen, Mr. William Henry",male,"(34.0, 40.0]",0,0,373450,"(7.854, 8.05]",Unknown,S,0,1


In [228]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,"(33.0, 41.0]",0,0,330911,"(7.796, 8.05]",Unknown,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,"(41.0, 48.0]",0,0,363272,"(-0.001, 7.644]",Unknown,S,3
2,894,2,"Myles, Mr. Thomas Francis",male,"(48.0, 76.0]",0,0,240276,"(8.05, 11.342]",Unknown,Q,1
3,895,3,"Wirz, Mr. Albert",male,"(25.0, 28.1]",0,0,315154,"(8.05, 11.342]",Unknown,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,"(21.0, 23.0]",0,1,3101298,"(11.342, 14.454]",Unknown,S,3


## Extract Cabin Number from Cabin

In [229]:
train['Cabin Num'] = train['Cabin'].str.extract(r'([0-9]+)').fillna(0).astype(float)
test['Cabin Num'] = test['Cabin'].str.extract(r'([0-9]+)').fillna(0).astype(float)

In [230]:
train['Cabin Class'] = train['Cabin'].str.extract(r'([A-Z]+)').fillna(0)
test['Cabin Class'] = test['Cabin'].str.extract(r'([A-Z]+)').fillna(0)

## Label Encoding

In [231]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title,Cabin Num,Cabin Class
0,1,0,3,"Braund, Mr. Owen Harris",male,"(20.0, 22.0]",0,0,A/5 21171,"(-0.001, 7.55]",Unknown,S,1,1,0.0,U
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,"(34.0, 40.0]",0,0,PC 17599,"(39.688, 77.958]",C85,C,4,3,85.0,C
2,3,1,3,"Heikkinen, Miss. Laina",female,"(25.0, 26.0]",0,0,STON/O2. 3101282,"(7.854, 8.05]",Unknown,S,3,2,0.0,U
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,"(34.0, 40.0]",0,0,113803,"(39.688, 77.958]",C123,S,3,3,123.0,C
4,5,0,3,"Allen, Mr. William Henry",male,"(34.0, 40.0]",0,0,373450,"(7.854, 8.05]",Unknown,S,0,1,0.0,U


In [232]:
#transfrom categorical data to numbers
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [233]:
# train.Name = train.Name.str.extract('([A-Za-z]+)\.', expand=False)
# test.Name = test.Name.str.extract('([A-Za-z]+)\.', expand=False)

In [234]:
train = train.drop('Name', axis=1)
test = test.drop('Name', axis=1)

In [235]:
train = train.drop('PassengerId', axis=1)
test = test.drop('PassengerId', axis=1)

In [236]:
train, test = label_encoding(train, test, ['Sex', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Age', 'Cabin Class'])

In [237]:
# pd.get_dummies(train['Fare'])

In [238]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title,Cabin Num,Cabin Class
0,0.0,3,1,4,0,0,720,0,186,2,1.0,1,0.0,8
1,1.0,1,0,15,0,0,816,15,106,0,4.0,3,85.0,2
2,1.0,3,0,9,0,0,914,5,186,2,3.0,2,0.0,8
3,1.0,1,0,15,0,0,65,15,70,2,3.0,3,123.0,2
4,0.0,3,1,15,0,0,649,5,186,2,0.0,1,0.0,8


In [240]:
# train.describe()

In [241]:
test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title,Cabin Num,Cabin Class
0,NaN,3,1,14,0,0,376,4,186,1,NaN,1,0.0,8
1,NaN,3,0,17,0,0,582,1,186,2,NaN,3,0.0,8
2,NaN,2,1,19,0,0,184,7,186,1,NaN,1,0.0,8
3,NaN,3,1,10,0,0,366,7,186,2,NaN,1,0.0,8
4,NaN,3,0,5,0,1,338,9,186,2,NaN,3,0.0,8


In [242]:
# test.describe()

# Building Model

## LGB

In [243]:
def optuna_lgb(n_trials=500):
    
    def objective(trial):

        params = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-5, 10, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-5, 10, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 62),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 0.9),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 0.9),
            "bagging_freq": trial.suggest_int("bagging_freq", 2, 9),
            "min_child_samples": trial.suggest_int("min_child_samples", 33, 93),
            "max_depth": trial.suggest_int("max_depth", 2, 7)
        }

        # initialize oof 
        oof_train = np.zeros((len(train),))

        for i in range(5):
            train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
            train_y = train.query(f'fold!={i}').Survived

            valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
            valid_y = train.query(f'fold=={i}').Survived
            
            lgb_train = lgb.Dataset(train_x,
                                    train_y)

            lgb_eval = lgb.Dataset(valid_x,
                                   valid_y,
                                   reference=lgb_train)

            gbm = lgb.train(params,
                            lgb_train,
                            valid_sets=[lgb_train, lgb_eval],
                            num_boost_round=5000,
                            verbose_eval=-1, 
                            early_stopping_rounds=1000
                            )
            oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
            oof_train[valid_x.index] = gbm.predict(valid_x,
                                                   num_iteration=gbm.best_iteration)
            
            
        return accuracy_score(train.Survived, oof_train.round())
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    best_params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }
    best_params.update(study.best_params)
    ## save trained model
    # initialize oof 
    oof_train = np.zeros((len(train),))
    print('*'*50)
    print('Exporting best models')
    print('*' * 50)
    # split folds using RSGKF
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        lgb_train = lgb.Dataset(train_x,
                                train_y)

        lgb_eval = lgb.Dataset(valid_x,
                               valid_y,
                               reference=lgb_train)

        gbm = lgb.train(best_params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_eval],
                        num_boost_round=50000,
                        verbose_eval=-1, 
                        early_stopping_rounds=1001
                        )

        oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
        oof_train[valid_x.index] = gbm.predict(valid_x, num_iteration=gbm.best_iteration)

        joblib.dump(gbm, f'lgb_fold{i}.pkl')

In [244]:
optuna_lgb() #0.8338945005611672(5 fold)  0.8451178451178452(10 fold)

[I 2021-10-03 20:57:08,497] A new study created in memory with name: no-name-b0cc7cfc-934c-4f65-9d10-274605a82c58


Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[416]	training's binary_logloss: 0.274603	valid_1's binary_logloss: 0.36191
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[128]	training's binary_logloss: 0.329816	valid_1's binary_logloss: 0.381863
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[162]	training's binary_logloss: 0.306457	valid_1's binary_logloss: 0.425746
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[79]	training's binary_logloss: 0.349277	valid_1's binary_logloss: 0.411182
Training until validation scores don't improve for 1000 rounds


[I 2021-10-03 20:57:10,029] Trial 0 finished with value: 0.8316498316498316 and parameters: {'lambda_l1': 1.4846135804465632, 'lambda_l2': 0.0011269705279818756, 'num_leaves': 58, 'feature_fraction': 0.8526920827636237, 'bagging_fraction': 0.6455359104190014, 'bagging_freq': 6, 'min_child_samples': 38, 'max_depth': 4}. Best is trial 0 with value: 0.8316498316498316.


Early stopping, best iteration is:
[70]	training's binary_logloss: 0.35905	valid_1's binary_logloss: 0.423843
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[695]	training's binary_logloss: 0.270551	valid_1's binary_logloss: 0.378479
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[209]	training's binary_logloss: 0.334154	valid_1's binary_logloss: 0.378051
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[305]	training's binary_logloss: 0.299271	valid_1's binary_logloss: 0.431071
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[318]	training's binary_logloss: 0.309439	valid_1's binary_logloss: 0.408664
Training until validation scores don't improve for 1000 rounds


[I 2021-10-03 20:57:11,501] Trial 1 finished with value: 0.835016835016835 and parameters: {'lambda_l1': 0.002656064985239344, 'lambda_l2': 0.04155069969841211, 'num_leaves': 39, 'feature_fraction': 0.6582692897835547, 'bagging_fraction': 0.8266025044788133, 'bagging_freq': 6, 'min_child_samples': 75, 'max_depth': 3}. Best is trial 1 with value: 0.835016835016835.


...

Early stopping, best iteration is:
[965]	training's binary_logloss: 0.230282	valid_1's binary_logloss: 0.419686
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[315]	training's binary_logloss: 0.313409	valid_1's binary_logloss: 0.377096
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[507]	training's binary_logloss: 0.276582	valid_1's binary_logloss: 0.37625
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[311]	training's binary_logloss: 0.304479	valid_1's binary_logloss: 0.419508
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[191]	training's binary_logloss: 0.332328	valid_1's binary_logloss: 0.41062
Training until validation scores don't improve for 1000 rounds


[I 2021-10-03 21:17:19,592] Trial 498 finished with value: 0.835016835016835 and parameters: {'lambda_l1': 0.0005072657343100868, 'lambda_l2': 0.0007404153743438929, 'num_leaves': 61, 'feature_fraction': 0.4721991650734731, 'bagging_fraction': 0.5456291968941342, 'bagging_freq': 5, 'min_child_samples': 53, 'max_depth': 7}. Best is trial 232 with value: 0.8540965207631874.


Early stopping, best iteration is:
[128]	training's binary_logloss: 0.35314	valid_1's binary_logloss: 0.423097
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[740]	training's binary_logloss: 0.273263	valid_1's binary_logloss: 0.36332
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[342]	training's binary_logloss: 0.316484	valid_1's binary_logloss: 0.378439
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[463]	training's binary_logloss: 0.287596	valid_1's binary_logloss: 0.428026
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[191]	training's binary_logloss: 0.336562	valid_1's binary_logloss: 0.409351
Training until validation scores don't improve for 1000 rounds


[I 2021-10-03 21:17:22,192] Trial 499 finished with value: 0.8462401795735129 and parameters: {'lambda_l1': 0.07121058978753904, 'lambda_l2': 0.00039219585460161346, 'num_leaves': 61, 'feature_fraction': 0.47155237309857967, 'bagging_fraction': 0.536724526015995, 'bagging_freq': 5, 'min_child_samples': 55, 'max_depth': 7}. Best is trial 232 with value: 0.8540965207631874.


Early stopping, best iteration is:
[128]	training's binary_logloss: 0.358933	valid_1's binary_logloss: 0.416812
Number of finished trials: 500
Best trial:
  Value: 0.8540965207631874
  Params: 
    lambda_l1: 0.003514432467253079
    lambda_l2: 5.578294918143333
    num_leaves: 61
    feature_fraction: 0.4553137353504819
    bagging_fraction: 0.5365347327520269
    bagging_freq: 5
    min_child_samples: 55
    max_depth: 7
**************************************************
Exporting best models
**************************************************
Training until validation scores don't improve for 1001 rounds
Early stopping, best iteration is:
[751]	training's binary_logloss: 0.305602	valid_1's binary_logloss: 0.364902
Training until validation scores don't improve for 1001 rounds
Early stopping, best iteration is:
[917]	training's binary_logloss: 0.287289	valid_1's binary_logloss: 0.371745
Training until validation scores don't improve for 1001 rounds
Early stopping, best iteration is:
[

## XGBoost

In [245]:
#optuna xgboost example
#https://github.com/optuna/optuna-examples/blob/main/xgboost/xgboost_integration.py
import xgboost as xgb
def objective(trial):
    param = {
            "verbosity": 0,
            "objective": "binary:logistic",  #binary:logistic
            "eval_metric": "auc",  #map
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        dtrain = xgb.DMatrix(train_x, label=train_y)
        dvalid = xgb.DMatrix(valid_x, label=valid_y)
        
        accuracy = []
        # Add a callback for pruning.
        pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
        bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
        preds = bst.predict(dvalid)
        pred_labels = np.rint(preds)
        accuracy.append(sklearn.metrics.accuracy_score(valid_y, pred_labels))
        
        
        joblib.dump(bst, f'xgboost_fold{i}.pkl')
    return np.mean(accuracy)

    
if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=500)
    print(study.best_trial)

[I 2021-10-03 21:17:26,173] A new study created in memory with name: no-name-955c05e5-a52c-4449-b772-1e1da69d3706


[0]	validation-auc:0.83010
[1]	validation-auc:0.84651
[2]	validation-auc:0.84737
[3]	validation-auc:0.84802
[4]	validation-auc:0.85020
[5]	validation-auc:0.85995
[6]	validation-auc:0.86621
[7]	validation-auc:0.86574
[8]	validation-auc:0.86614
[9]	validation-auc:0.86601
[0]	validation-auc:0.84311
[1]	validation-auc:0.84572
[2]	validation-auc:0.84927
[3]	validation-auc:0.84652
[4]	validation-auc:0.84405
[5]	validation-auc:0.84318
[6]	validation-auc:0.84672
[7]	validation-auc:0.84753
[8]	validation-auc:0.84726
[9]	validation-auc:0.84606
[0]	validation-auc:0.83917
[1]	validation-auc:0.83489
[2]	validation-auc:0.83322
[3]	validation-auc:0.83402
[4]	validation-auc:0.83135
[5]	validation-auc:0.83068
[6]	validation-auc:0.82968
[7]	validation-auc:0.82948
[8]	validation-auc:0.82808
[9]	validation-auc:0.82687
[0]	validation-auc:0.80755
[1]	validation-auc:0.81243
[2]	validation-auc:0.81324
[3]	validation-auc:0.81417
[4]	validation-auc:0.81337
[5]	validation-auc:0.81377
[6]	validation-auc:0.81678
[

[I 2021-10-03 21:17:26,675] Trial 0 finished with value: 0.7696629213483146 and parameters: {'booster': 'gbtree', 'lambda': 1.6575558284252807e-07, 'alpha': 7.876790528044266e-07, 'max_depth': 9, 'eta': 0.00011059920100539784, 'gamma': 0.5133030102721596, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.7696629213483146.


[0]	validation-auc:0.85823
[1]	validation-auc:0.87312
[2]	validation-auc:0.88485
[3]	validation-auc:0.88050
[4]	validation-auc:0.88142
[5]	validation-auc:0.89229
[6]	validation-auc:0.89124
[7]	validation-auc:0.89170
[8]	validation-auc:0.89974
[9]	validation-auc:0.90178
[0]	validation-auc:0.88336
[1]	validation-auc:0.86230
[2]	validation-auc:0.84459
[3]	validation-auc:0.85421
[4]	validation-auc:0.86671
[5]	validation-auc:0.85949
[6]	validation-auc:0.86852
[7]	validation-auc:0.86183
[8]	validation-auc:0.85174
[9]	validation-auc:0.86083
[0]	validation-auc:0.81979
[1]	validation-auc:0.81364
[2]	validation-auc:0.83583
[3]	validation-auc:0.83536
[4]	validation-auc:0.84993
[5]	validation-auc:0.84693
[6]	validation-auc:0.84906
[7]	validation-auc:0.85481
[8]	validation-auc:0.84987
[9]	validation-auc:0.84251
[0]	validation-auc:0.85134
[1]	validation-auc:0.85000
[2]	validation-auc:0.87253
[3]	validation-auc:0.87400
[4]	validation-auc:0.87955
[5]	validation-auc:0.87741
[6]	validation-auc:0.88041
[

[I 2021-10-03 21:17:27,171] Trial 1 finished with value: 0.8146067415730337 and parameters: {'booster': 'dart', 'lambda': 0.006938420659306094, 'alpha': 2.2214784621160383e-08, 'max_depth': 6, 'eta': 0.18965135112581485, 'gamma': 0.004704878168178699, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.01942409421128637, 'skip_drop': 0.06379761917940067}. Best is trial 1 with value: 0.8146067415730337.


...

[0]	validation-auc:0.84084
[1]	validation-auc:0.85448
[2]	validation-auc:0.85448
[3]	validation-auc:0.85448
[4]	validation-auc:0.85448


[I 2021-10-03 21:19:59,291] Trial 498 pruned. Trial was pruned at iteration 5.


[0]	validation-auc:0.88656
[1]	validation-auc:0.88827
[2]	validation-auc:0.88801
[3]	validation-auc:0.88577
[4]	validation-auc:0.88801
[5]	validation-auc:0.88801
[6]	validation-auc:0.88801
[7]	validation-auc:0.88577
[8]	validation-auc:0.88801
[9]	validation-auc:0.88801
[0]	validation-auc:0.83302
[1]	validation-auc:0.83302
[2]	validation-auc:0.83302
[3]	validation-auc:0.83302
[4]	validation-auc:0.83302
[5]	validation-auc:0.83302
[6]	validation-auc:0.83302
[7]	validation-auc:0.83302
[8]	validation-auc:0.83302
[9]	validation-auc:0.83302
[0]	validation-auc:0.84672
[1]	validation-auc:0.84672
[2]	validation-auc:0.84672
[3]	validation-auc:0.84672
[4]	validation-auc:0.84672
[5]	validation-auc:0.84672
[6]	validation-auc:0.84672
[7]	validation-auc:0.84672
[8]	validation-auc:0.84672
[9]	validation-auc:0.84672
[0]	validation-auc:0.85314
[1]	validation-auc:0.85314
[2]	validation-auc:0.85314
[3]	validation-auc:0.85314
[4]	validation-auc:0.85314
[5]	validation-auc:0.85314
[6]	validation-auc:0.85314
[

[I 2021-10-03 21:19:59,840] Trial 499 finished with value: 0.8314606741573034 and parameters: {'booster': 'dart', 'lambda': 3.090659086638229e-08, 'alpha': 3.2415319577374386e-08, 'max_depth': 3, 'eta': 0.001697462271264485, 'gamma': 9.78614316456531e-05, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.0015332776648907735, 'skip_drop': 0.28294988749510597}. Best is trial 7 with value: 0.8314606741573034.


FrozenTrial(number=7, values=[0.8314606741573034], datetime_start=datetime.datetime(2021, 10, 3, 21, 17, 28, 509538), datetime_complete=datetime.datetime(2021, 10, 3, 21, 17, 28, 918699), params={'booster': 'dart', 'lambda': 0.0009913304968890962, 'alpha': 0.017280214240420012, 'max_depth': 3, 'eta': 2.0336862132443787e-07, 'gamma': 0.02220699824114986, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 1.2514757565514473e-05, 'skip_drop': 7.139213398719718e-08}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08), 'max_depth': IntUniformDistribution(high=9, low=1, step=1), 'eta': LogUniformDistribution(high=1.0, low=1e-08), 'gamma': LogUniformDistribution(high=1.0, low=1e-08), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide')), 'sample_type': CategoricalDistribution(choices

## Catboost

In [246]:
#optuna catboost example
#https://github.com/optuna/optuna-examples/blob/main/catboost/catboost_simple.py
import catboost as cb
def objective(trial):
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
        
        #save the five fold accuracy
        accuracy = []
        
        gbm = cb.CatBoostClassifier(**param)
        gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
        
        #save the best model for each fold 
        joblib.dump(gbm, f'catboost_fold{i}.pkl')
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=500, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-03 21:19:59,867] A new study created in memory with name: no-name-f778e6fb-7ccf-4bab-83e9-16eecc1d946f
[I 2021-10-03 21:20:03,643] Trial 0 finished with value: 0.8202247191011236 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.027591314192482418, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.8202247191011236.
[I 2021-10-03 21:20:06,720] Trial 1 finished with value: 0.8258426966292135 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.020703271611936486, 'depth': 8, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 7.317612581144352}. Best is trial 1 with value: 0.8258426966292135.
[I 2021-10-03 21:20:11,784] Trial 2 finished with value: 0.8370786516853933 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.06823403495829282, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 2.6323058699056845}. Best is tria

[I 2021-10-03 21:21:54,011] Trial 27 finished with value: 0.848314606741573 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.09499944232799011, 'depth': 5, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.7132858624813858}. Best is trial 26 with value: 0.8595505617977528.
[I 2021-10-03 21:21:58,450] Trial 28 finished with value: 0.7415730337078652 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.010015311734086901, 'depth': 11, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 26 with value: 0.8595505617977528.
[I 2021-10-03 21:22:01,166] Trial 29 finished with value: 0.8202247191011236 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.08539184426297938, 'depth': 1, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.3646988189200898}. Best is trial 26 with value: 0.8595505617977528.
[I 2021-10-03 21:22:04,227] Trial 30 finished with value: 0.848314606741573 and parameters: {'o

[I 2021-10-03 21:23:56,909] Trial 54 finished with value: 0.8595505617977528 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.07766052874272436, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.011455480336183}. Best is trial 26 with value: 0.8595505617977528.
[I 2021-10-03 21:24:02,163] Trial 55 finished with value: 0.8370786516853933 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.08383659684131804, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.4087865216005175}. Best is trial 26 with value: 0.8595505617977528.
[I 2021-10-03 21:24:06,270] Trial 56 finished with value: 0.8370786516853933 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.05757390714192774, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 26 with value: 0.8595505617977528.
[I 2021-10-03 21:24:11,695] Trial 57 finished with value: 0.848314606741573 an

[I 2021-10-03 21:26:17,551] Trial 82 finished with value: 0.848314606741573 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.08969087259528094, 'depth': 5, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 68 with value: 0.8764044943820225.
[I 2021-10-03 21:26:22,382] Trial 83 finished with value: 0.848314606741573 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.09973422747665812, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 68 with value: 0.8764044943820225.
[I 2021-10-03 21:26:26,403] Trial 84 finished with value: 0.8595505617977528 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.08150946297439206, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 68 with value: 0.8764044943820225.
[I 2021-10-03 21:26:30,494] Trial 85 finished with value: 0.8426966292134831 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.07521347066725088, 'depth': 7, 'boos

[I 2021-10-03 21:28:37,701] Trial 111 finished with value: 0.8539325842696629 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.058054508362042755, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 68 with value: 0.8764044943820225.
[I 2021-10-03 21:28:42,481] Trial 112 finished with value: 0.848314606741573 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.09045503282252078, 'depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 68 with value: 0.8764044943820225.
[I 2021-10-03 21:28:47,530] Trial 113 finished with value: 0.8595505617977528 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.09244307181029306, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 68 with value: 0.8764044943820225.
[I 2021-10-03 21:28:52,979] Trial 114 finished with value: 0.8595505617977528 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.08565624436536912, 'depth': 1

Number of finished trials: 129
Best trial:
  Value: 0.8764044943820225
  Params: 
    objective: Logloss
    colsample_bylevel: 0.09580573043166755
    depth: 6
    boosting_type: Ordered
    bootstrap_type: MVS


## Logistic and Random Forest and SVC

In [ ]:
# #Step 1. Define an objective function to be maximized.
# def objective(trial):

#     classifier_name = trial.suggest_categorical("classifier", ["LogReg", "SVC", "RandomForest"])
    
#     # Step 2. Setup values for the hyperparameters:
#     if classifier_name == 'LogReg':
#         logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
#         classifier_obj = linear_model.LogisticRegression(C=logreg_c)
#     elif classifier_name == "SVC":
#         svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
#         classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
#     else:
#         rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
#         rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
#         classifier_obj = ensemble.RandomForestClassifier(
#             max_depth=rf_max_depth, n_estimators=rf_n_estimators
#         )
        
#     for i in range(5):
#         train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
#         train_y = train.query(f'fold!={i}').Survived

#         valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
#         valid_y = train.query(f'fold=={i}').Survived
    
#         #save the five fold accuracy
#         accuracy = []

#         gbm = classifier_obj
#         gbm.fit(train_x, train_y)
#         preds = gbm.predict(valid_x)
#         pred_labels = np.rint(preds)
#         accuracy.append(accuracy_score(valid_y, pred_labels))
                
#         #save the best model for each fold 
#         joblib.dump(gbm, f'log_randomf_fold{i}.pkl')
        
#     return np.mean(accuracy)

# if __name__ == "__main__":
#     study = optuna.create_study(direction="maximize")
#     study.optimize(objective, n_trials=100, timeout=600)

#     print("Number of finished trials: {}".format(len(study.trials)))

#     print("Best trial:")
#     trial = study.best_trial

#     print("  Value: {}".format(trial.value))

#     print("  Params: ")
#     for key, value in trial.params.items():
#         print("    {}: {}".format(key, value))

## Logistic

In [247]:
#logistic

#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = 'LogReg'
    logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
    classifier_obj = linear_model.LogisticRegression(C=logreg_c)
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'logistic_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=500, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-03 21:30:03,747] A new study created in memory with name: no-name-57d605de-213e-41b8-8a5b-30fe27ef83e2
[I 2021-10-03 21:30:04,013] Trial 0 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 12.027281429720619}. Best is trial 0 with value: 0.7865168539325843.
[I 2021-10-03 21:30:04,229] Trial 1 finished with value: 0.7696629213483146 and parameters: {'logreg_c': 1050575.1829994204}. Best is trial 0 with value: 0.7865168539325843.
[I 2021-10-03 21:30:04,457] Trial 2 finished with value: 0.7808988764044944 and parameters: {'logreg_c': 3821.7948577825996}. Best is trial 0 with value: 0.7865168539325843.
[I 2021-10-03 21:30:04,662] Trial 3 finished with value: 0.7696629213483146 and parameters: {'logreg_c': 2015247084.1806157}. Best is trial 0 with value: 0.7865168539325843.
[I 2021-10-03 21:30:04,825] Trial 4 finished with value: 0.6853932584269663 and parameters: {'logreg_c': 0.0016599088854736634}. Best is trial 0 with value: 0.7865168539325843.
[I 2021-10-03

[I 2021-10-03 21:30:12,280] Trial 43 finished with value: 0.8033707865168539 and parameters: {'logreg_c': 25.3756244512754}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:12,521] Trial 44 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 333.1300286136832}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:12,733] Trial 45 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 0.1351108380014819}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:12,963] Trial 46 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 35.62946017226727}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:13,204] Trial 47 finished with value: 0.6910112359550562 and parameters: {'logreg_c': 0.0032276956163818453}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:13,420] Trial 48 finished with value: 0.7808988764044944 and parameters: {'logreg_c': 2.317541976918753}. 

[I 2021-10-03 21:30:21,873] Trial 87 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 2106.6905035320087}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:22,119] Trial 88 finished with value: 0.6853932584269663 and parameters: {'logreg_c': 0.0009404677570314685}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:22,406] Trial 89 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 20.18198842231527}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:22,689] Trial 90 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 2.2674418706979496}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:22,985] Trial 91 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 5.584760017010755}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:23,185] Trial 92 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 61.06335861683083}

[I 2021-10-03 21:30:31,047] Trial 131 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 24.950036578010817}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:31,245] Trial 132 finished with value: 0.8089887640449438 and parameters: {'logreg_c': 2.2198800394434413}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:31,445] Trial 133 finished with value: 0.8033707865168539 and parameters: {'logreg_c': 2.707180340910588}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:31,640] Trial 134 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 0.3552705561029012}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:31,838] Trial 135 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 0.7988857176024764}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:32,035] Trial 136 finished with value: 0.7752808988764045 and parameters: {'logreg_c': 0.074548483766

[I 2021-10-03 21:30:39,741] Trial 175 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 0.115369834246788}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:39,932] Trial 176 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 14.485251913058201}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:40,132] Trial 177 finished with value: 0.797752808988764 and parameters: {'logreg_c': 3.162680663814248}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:40,326] Trial 178 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 61.546352094766}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:40,520] Trial 179 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 0.3126399008662991}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:30:40,719] Trial 180 finished with value: 0.7808988764044944 and parameters: {'logreg_c': 297.2849923085357}.

...

[I 2021-10-03 21:31:15,068] Trial 351 finished with value: 0.8033707865168539 and parameters: {'logreg_c': 38.38241488046984}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:15,260] Trial 352 finished with value: 0.8033707865168539 and parameters: {'logreg_c': 1.4748810620213162}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:15,448] Trial 353 finished with value: 0.7808988764044944 and parameters: {'logreg_c': 2.952942421876397}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:15,639] Trial 354 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 88.71242187059852}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:15,830] Trial 355 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 19.10382730367779}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:16,023] Trial 356 finished with value: 0.7808988764044944 and parameters: {'logreg_c': 2.386247485438374

[I 2021-10-03 21:31:23,536] Trial 395 finished with value: 0.7865168539325843 and parameters: {'logreg_c': 4.981344000766328}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:23,720] Trial 396 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 0.9724112936630349}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:23,912] Trial 397 finished with value: 0.7752808988764045 and parameters: {'logreg_c': 2.50091363225409}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:24,102] Trial 398 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 17.37063052236673}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:24,290] Trial 399 finished with value: 0.8146067415730337 and parameters: {'logreg_c': 4.810718747640242}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:24,481] Trial 400 finished with value: 0.8033707865168539 and parameters: {'logreg_c': 23.85437371793047}

[I 2021-10-03 21:31:32,057] Trial 439 finished with value: 0.797752808988764 and parameters: {'logreg_c': 7.929958972796851}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:32,249] Trial 440 finished with value: 0.797752808988764 and parameters: {'logreg_c': 0.06948638770156051}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:32,441] Trial 441 finished with value: 0.7696629213483146 and parameters: {'logreg_c': 2.2518474596226077}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:32,631] Trial 442 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 248.48913462779873}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:32,821] Trial 443 finished with value: 0.7921348314606742 and parameters: {'logreg_c': 14.447821615123654}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:33,014] Trial 444 finished with value: 0.797752808988764 and parameters: {'logreg_c': 5.81261274592186

[I 2021-10-03 21:31:40,505] Trial 483 finished with value: 0.8089887640449438 and parameters: {'logreg_c': 13.547200315122586}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:40,695] Trial 484 finished with value: 0.8033707865168539 and parameters: {'logreg_c': 2.261745147078145}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:40,892] Trial 485 finished with value: 0.8089887640449438 and parameters: {'logreg_c': 0.6759938137640588}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:41,087] Trial 486 finished with value: 0.797752808988764 and parameters: {'logreg_c': 2.5725299590063995}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:41,280] Trial 487 finished with value: 0.7696629213483146 and parameters: {'logreg_c': 0.13581400261759743}. Best is trial 9 with value: 0.8258426966292135.
[I 2021-10-03 21:31:41,477] Trial 488 finished with value: 0.8146067415730337 and parameters: {'logreg_c': 10.08772410634

Number of finished trials: 500
Best trial:
  Value: 0.8258426966292135
  Params: 
    logreg_c: 4.400063774907226


## SVC

In [248]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    # Step 2. Setup values for the hyperparameters:
    classifier_name = "SVC"
    svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
    classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'svc_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=500, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-03 21:31:43,546] A new study created in memory with name: no-name-9001a94c-a8aa-482e-8e39-de6450877941
[I 2021-10-03 21:31:43,751] Trial 0 finished with value: 0.6910112359550562 and parameters: {'svc_c': 140007.65067668265}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:43,918] Trial 1 finished with value: 0.6292134831460674 and parameters: {'svc_c': 0.4941285991828142}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:44,085] Trial 2 finished with value: 0.6123595505617978 and parameters: {'svc_c': 0.24048623879092623}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:44,218] Trial 3 finished with value: 0.6123595505617978 and parameters: {'svc_c': 1.0586331243558424e-06}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:44,401] Trial 4 finished with value: 0.6910112359550562 and parameters: {'svc_c': 4819.141369102905}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:44,553]

[I 2021-10-03 21:31:51,349] Trial 44 finished with value: 0.6910112359550562 and parameters: {'svc_c': 407716949.2689881}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:51,507] Trial 45 finished with value: 0.6629213483146067 and parameters: {'svc_c': 0.7633617143302129}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:51,678] Trial 46 finished with value: 0.6910112359550562 and parameters: {'svc_c': 3367.1823761781143}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:51,849] Trial 47 finished with value: 0.6910112359550562 and parameters: {'svc_c': 1255715640.249682}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:52,021] Trial 48 finished with value: 0.6910112359550562 and parameters: {'svc_c': 9.097911946299336}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:52,193] Trial 49 finished with value: 0.6910112359550562 and parameters: {'svc_c': 1445840308.8422272}. Best is trial 0 wit

[I 2021-10-03 21:31:58,698] Trial 89 finished with value: 0.6910112359550562 and parameters: {'svc_c': 2044.774518753378}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:58,855] Trial 90 finished with value: 0.6910112359550562 and parameters: {'svc_c': 828.186258536571}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:59,011] Trial 91 finished with value: 0.6910112359550562 and parameters: {'svc_c': 7850.394347375203}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:59,168] Trial 92 finished with value: 0.6910112359550562 and parameters: {'svc_c': 13792.092123252572}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:59,324] Trial 93 finished with value: 0.6910112359550562 and parameters: {'svc_c': 135.52448306950006}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:31:59,482] Trial 94 finished with value: 0.6910112359550562 and parameters: {'svc_c': 440.5158719020592}. Best is trial 0 with 

[I 2021-10-03 21:32:05,619] Trial 134 finished with value: 0.6910112359550562 and parameters: {'svc_c': 768.6737552123733}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:05,770] Trial 135 finished with value: 0.6910112359550562 and parameters: {'svc_c': 31465.316756505978}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:05,920] Trial 136 finished with value: 0.6910112359550562 and parameters: {'svc_c': 6009.463265649009}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:06,070] Trial 137 finished with value: 0.6910112359550562 and parameters: {'svc_c': 48.92414603950969}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:06,222] Trial 138 finished with value: 0.6910112359550562 and parameters: {'svc_c': 92.99432087077868}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:06,372] Trial 139 finished with value: 0.6910112359550562 and parameters: {'svc_c': 34.34603720900329}. Best is trial 0

[I 2021-10-03 21:32:12,298] Trial 179 finished with value: 0.6910112359550562 and parameters: {'svc_c': 1513.938087400834}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:12,446] Trial 180 finished with value: 0.6910112359550562 and parameters: {'svc_c': 2829.0161624627312}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:12,590] Trial 181 finished with value: 0.6910112359550562 and parameters: {'svc_c': 18397.80751692456}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:12,738] Trial 182 finished with value: 0.6910112359550562 and parameters: {'svc_c': 30096.7016291732}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:12,880] Trial 183 finished with value: 0.6910112359550562 and parameters: {'svc_c': 559478.6300095653}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:13,026] Trial 184 finished with value: 0.6910112359550562 and parameters: {'svc_c': 2886.692252627374}. Best is trial 0 

...

[I 2021-10-03 21:32:38,177] Trial 359 finished with value: 0.6910112359550562 and parameters: {'svc_c': 159449968.24055004}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:38,317] Trial 360 finished with value: 0.6910112359550562 and parameters: {'svc_c': 735145565.9957708}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:38,460] Trial 361 finished with value: 0.6910112359550562 and parameters: {'svc_c': 1721392621.0258465}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:38,601] Trial 362 finished with value: 0.6910112359550562 and parameters: {'svc_c': 25266955.228402026}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:38,741] Trial 363 finished with value: 0.6910112359550562 and parameters: {'svc_c': 39163285.483005345}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:38,880] Trial 364 finished with value: 0.6910112359550562 and parameters: {'svc_c': 508894014.6136642}. Best is tria

[I 2021-10-03 21:32:44,538] Trial 404 finished with value: 0.6910112359550562 and parameters: {'svc_c': 989.3192976761487}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:44,680] Trial 405 finished with value: 0.6910112359550562 and parameters: {'svc_c': 354.6194357214565}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:44,822] Trial 406 finished with value: 0.6910112359550562 and parameters: {'svc_c': 634.1766613438152}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:44,960] Trial 407 finished with value: 0.6910112359550562 and parameters: {'svc_c': 215.20298734983925}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:45,104] Trial 408 finished with value: 0.6910112359550562 and parameters: {'svc_c': 3101.0791904181683}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:45,244] Trial 409 finished with value: 0.6910112359550562 and parameters: {'svc_c': 630.4177159779107}. Best is trial 

[I 2021-10-03 21:32:50,936] Trial 449 finished with value: 0.6910112359550562 and parameters: {'svc_c': 14.45999835245166}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:51,077] Trial 450 finished with value: 0.6910112359550562 and parameters: {'svc_c': 6.908710039863985}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:51,218] Trial 451 finished with value: 0.6910112359550562 and parameters: {'svc_c': 2.0523696926391435}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:51,359] Trial 452 finished with value: 0.6910112359550562 and parameters: {'svc_c': 42.53519639259584}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:51,501] Trial 453 finished with value: 0.6910112359550562 and parameters: {'svc_c': 3.2186472086826283}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:51,630] Trial 454 finished with value: 0.6292134831460674 and parameters: {'svc_c': 0.5071533475782399}. Best is trial

[I 2021-10-03 21:32:57,193] Trial 494 finished with value: 0.6910112359550562 and parameters: {'svc_c': 144651.33509420106}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:57,337] Trial 495 finished with value: 0.6910112359550562 and parameters: {'svc_c': 287414.1724572892}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:57,479] Trial 496 finished with value: 0.6910112359550562 and parameters: {'svc_c': 93707.71286069781}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:57,621] Trial 497 finished with value: 0.6910112359550562 and parameters: {'svc_c': 243484.762946296}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:57,762] Trial 498 finished with value: 0.6910112359550562 and parameters: {'svc_c': 304913.531216248}. Best is trial 0 with value: 0.6910112359550562.
[I 2021-10-03 21:32:57,906] Trial 499 finished with value: 0.6910112359550562 and parameters: {'svc_c': 88616.4885059558}. Best is trial 0 wi

Number of finished trials: 500
Best trial:
  Value: 0.6910112359550562
  Params: 
    svc_c: 140007.65067668265


## RandomForest

In [249]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = 'RandomForest'
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
    rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
#     rf_min_depth = trial.suggest_int("rf_min_depth", 2, 32, log=True)
    classifier_obj = ensemble.RandomForestClassifier(
        max_depth=rf_max_depth, n_estimators=rf_n_estimators
    )
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'rf_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=500, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-03 21:32:57,933] A new study created in memory with name: no-name-39bcab1b-490d-49c4-a914-6498671656df
[I 2021-10-03 21:33:04,074] Trial 0 finished with value: 0.8370786516853933 and parameters: {'rf_n_estimators': 821, 'rf_max_depth': 4}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:33:08,200] Trial 1 finished with value: 0.8033707865168539 and parameters: {'rf_n_estimators': 567, 'rf_max_depth': 2}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:33:10,670] Trial 2 finished with value: 0.8314606741573034 and parameters: {'rf_n_estimators': 347, 'rf_max_depth': 2}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:33:18,555] Trial 3 finished with value: 0.8426966292134831 and parameters: {'rf_n_estimators': 979, 'rf_max_depth': 10}. Best is trial 3 with value: 0.8426966292134831.
[I 2021-10-03 21:33:24,432] Trial 4 finished with value: 0.8539325842696629 and parameters: {'rf_n_estimators': 713, 'rf_max_depth': 12}. Best is

[I 2021-10-03 21:36:53,322] Trial 41 finished with value: 0.848314606741573 and parameters: {'rf_n_estimators': 640, 'rf_max_depth': 11}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:36:59,719] Trial 42 finished with value: 0.8426966292134831 and parameters: {'rf_n_estimators': 778, 'rf_max_depth': 11}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:37:05,356] Trial 43 finished with value: 0.8426966292134831 and parameters: {'rf_n_estimators': 672, 'rf_max_depth': 15}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:37:10,091] Trial 44 finished with value: 0.8426966292134831 and parameters: {'rf_n_estimators': 606, 'rf_max_depth': 6}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:37:14,454] Trial 45 finished with value: 0.848314606741573 and parameters: {'rf_n_estimators': 520, 'rf_max_depth': 17}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:37:19,742] Trial 46 finished with value: 0.83707865

[I 2021-10-03 21:41:05,242] Trial 83 finished with value: 0.8539325842696629 and parameters: {'rf_n_estimators': 728, 'rf_max_depth': 12}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:41:11,937] Trial 84 finished with value: 0.8426966292134831 and parameters: {'rf_n_estimators': 777, 'rf_max_depth': 9}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:41:17,789] Trial 85 finished with value: 0.848314606741573 and parameters: {'rf_n_estimators': 656, 'rf_max_depth': 11}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:41:22,854] Trial 86 finished with value: 0.848314606741573 and parameters: {'rf_n_estimators': 557, 'rf_max_depth': 14}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:41:30,066] Trial 87 finished with value: 0.8426966292134831 and parameters: {'rf_n_estimators': 816, 'rf_max_depth': 15}. Best is trial 4 with value: 0.8539325842696629.
[I 2021-10-03 21:41:35,932] Trial 88 finished with value: 0.84269662

Number of finished trials: 102
Best trial:
  Value: 0.8539325842696629
  Params: 
    rf_n_estimators: 713
    rf_max_depth: 12


## Decision Tree

In [250]:
#Step 1. Define an objective function to be maximized.
def objective(trial):
    max_depth = trial.suggest_int("max_depth", 2, 612)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 612)
    max_leaf_nodes = int(trial.suggest_int("max_leaf_nodes", 2, 612))
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    DTC = DecisionTreeClassifier(min_samples_split = min_samples_split, 
                                 max_leaf_nodes = max_leaf_nodes,
                                 criterion = criterion)
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        DTC.fit(train_x, train_y)
        preds = DTC.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(DTC, f'dt_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=500, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-03 21:43:01,488] A new study created in memory with name: no-name-2a239bca-d0bf-4b5f-8ae4-8f76c0a91177
[I 2021-10-03 21:43:01,561] Trial 0 finished with value: 0.8370786516853933 and parameters: {'max_depth': 585, 'min_samples_split': 127, 'max_leaf_nodes': 97, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:01,625] Trial 1 finished with value: 0.8033707865168539 and parameters: {'max_depth': 292, 'min_samples_split': 349, 'max_leaf_nodes': 268, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:01,691] Trial 2 finished with value: 0.8033707865168539 and parameters: {'max_depth': 555, 'min_samples_split': 302, 'max_leaf_nodes': 209, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:01,760] Trial 3 finished with value: 0.7865168539325843 and parameters: {'max_depth': 103, 'min_samples_split': 265, 'max_leaf_nodes': 28, 'criterion': 'gini'}. Best is

[I 2021-10-03 21:43:03,979] Trial 33 finished with value: 0.7865168539325843 and parameters: {'max_depth': 46, 'min_samples_split': 244, 'max_leaf_nodes': 519, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:04,055] Trial 34 finished with value: 0.8033707865168539 and parameters: {'max_depth': 125, 'min_samples_split': 328, 'max_leaf_nodes': 469, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:04,132] Trial 35 finished with value: 0.7865168539325843 and parameters: {'max_depth': 56, 'min_samples_split': 287, 'max_leaf_nodes': 560, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:04,218] Trial 36 finished with value: 0.7865168539325843 and parameters: {'max_depth': 10, 'min_samples_split': 172, 'max_leaf_nodes': 544, 'criterion': 'gini'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:04,288] Trial 37 finished with value: 0.8314606741

[I 2021-10-03 21:43:06,411] Trial 67 finished with value: 0.8370786516853933 and parameters: {'max_depth': 586, 'min_samples_split': 129, 'max_leaf_nodes': 579, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:06,478] Trial 68 finished with value: 0.7865168539325843 and parameters: {'max_depth': 528, 'min_samples_split': 253, 'max_leaf_nodes': 411, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:06,545] Trial 69 finished with value: 0.7865168539325843 and parameters: {'max_depth': 377, 'min_samples_split': 168, 'max_leaf_nodes': 230, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:06,615] Trial 70 finished with value: 0.8202247191011236 and parameters: {'max_depth': 610, 'min_samples_split': 54, 'max_leaf_nodes': 289, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:06,684] Trial 71 finished with value: 0.79213

[I 2021-10-03 21:43:08,842] Trial 101 finished with value: 0.8314606741573034 and parameters: {'max_depth': 511, 'min_samples_split': 115, 'max_leaf_nodes': 569, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:08,909] Trial 102 finished with value: 0.8202247191011236 and parameters: {'max_depth': 86, 'min_samples_split': 64, 'max_leaf_nodes': 601, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:08,976] Trial 103 finished with value: 0.8370786516853933 and parameters: {'max_depth': 555, 'min_samples_split': 121, 'max_leaf_nodes': 534, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:09,044] Trial 104 finished with value: 0.8370786516853933 and parameters: {'max_depth': 612, 'min_samples_split': 135, 'max_leaf_nodes': 384, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:09,114] Trial 105 finished with value: 0.8

[I 2021-10-03 21:43:11,147] Trial 135 finished with value: 0.8033707865168539 and parameters: {'max_depth': 495, 'min_samples_split': 477, 'max_leaf_nodes': 515, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:11,213] Trial 136 finished with value: 0.8033707865168539 and parameters: {'max_depth': 568, 'min_samples_split': 157, 'max_leaf_nodes': 506, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:11,281] Trial 137 finished with value: 0.8314606741573034 and parameters: {'max_depth': 608, 'min_samples_split': 116, 'max_leaf_nodes': 388, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:11,351] Trial 138 finished with value: 0.7921348314606742 and parameters: {'max_depth': 582, 'min_samples_split': 104, 'max_leaf_nodes': 557, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:11,417] Trial 139 finished with value: 0

...

[I 2021-10-03 21:43:27,786] Trial 373 finished with value: 0.8370786516853933 and parameters: {'max_depth': 253, 'min_samples_split': 133, 'max_leaf_nodes': 11, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:27,853] Trial 374 finished with value: 0.8033707865168539 and parameters: {'max_depth': 549, 'min_samples_split': 304, 'max_leaf_nodes': 55, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:27,920] Trial 375 finished with value: 0.8370786516853933 and parameters: {'max_depth': 252, 'min_samples_split': 127, 'max_leaf_nodes': 32, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:27,993] Trial 376 finished with value: 0.8033707865168539 and parameters: {'max_depth': 495, 'min_samples_split': 149, 'max_leaf_nodes': 66, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:28,065] Trial 377 finished with value: 0.837

[I 2021-10-03 21:43:30,239] Trial 407 finished with value: 0.7865168539325843 and parameters: {'max_depth': 591, 'min_samples_split': 167, 'max_leaf_nodes': 523, 'criterion': 'gini'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:30,316] Trial 408 finished with value: 0.8033707865168539 and parameters: {'max_depth': 558, 'min_samples_split': 150, 'max_leaf_nodes': 533, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:30,387] Trial 409 finished with value: 0.8370786516853933 and parameters: {'max_depth': 559, 'min_samples_split': 120, 'max_leaf_nodes': 27, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:30,455] Trial 410 finished with value: 0.7921348314606742 and parameters: {'max_depth': 570, 'min_samples_split': 104, 'max_leaf_nodes': 77, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:30,523] Trial 411 finished with value: 0.8033

[I 2021-10-03 21:43:32,612] Trial 441 finished with value: 0.8033707865168539 and parameters: {'max_depth': 6, 'min_samples_split': 139, 'max_leaf_nodes': 261, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:32,680] Trial 442 finished with value: 0.7865168539325843 and parameters: {'max_depth': 169, 'min_samples_split': 172, 'max_leaf_nodes': 378, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:32,747] Trial 443 finished with value: 0.8370786516853933 and parameters: {'max_depth': 343, 'min_samples_split': 132, 'max_leaf_nodes': 443, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:32,822] Trial 444 finished with value: 0.8370786516853933 and parameters: {'max_depth': 517, 'min_samples_split': 120, 'max_leaf_nodes': 37, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:32,894] Trial 445 finished with value: 0.79

[I 2021-10-03 21:43:35,015] Trial 475 finished with value: 0.8370786516853933 and parameters: {'max_depth': 44, 'min_samples_split': 122, 'max_leaf_nodes': 558, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:35,089] Trial 476 finished with value: 0.7921348314606742 and parameters: {'max_depth': 501, 'min_samples_split': 102, 'max_leaf_nodes': 46, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:35,160] Trial 477 finished with value: 0.8370786516853933 and parameters: {'max_depth': 508, 'min_samples_split': 119, 'max_leaf_nodes': 551, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:35,228] Trial 478 finished with value: 0.8370786516853933 and parameters: {'max_depth': 492, 'min_samples_split': 133, 'max_leaf_nodes': 408, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8370786516853933.
[I 2021-10-03 21:43:35,300] Trial 479 finished with value: 0.8

Number of finished trials: 500
Best trial:
  Value: 0.8370786516853933
  Params: 
    max_depth: 585
    min_samples_split: 127
    max_leaf_nodes: 97
    criterion: entropy


## Ensemble Model

In [253]:
X_train = train.drop(['Survived', 'fold'], axis=1).values
y_train = train[['Survived']].values

In [254]:
test2 = test.drop(['Survived', 'fold'], axis=1).values

In [255]:
#get the mean of best fold in lgb   0.8338945005611672  0.77990 selected  
                                        # 0.8540965207631874 (trials=500)
lgb_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./lgb_fold{i}.pkl')
    lgb_pred += model.predict(test2) / 5
    
lgb_pred = lgb_pred.round().astype(int)

In [256]:
#get the mean of best fold in xgboost  0.8314606741573034  0.78468 selected 
                                        # 0.8314606741573034（500）
from xgboost import XGBClassifier
xgboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./xgboost_fold{i}.pkl')
    model = XGBClassifier()
    model.fit(X_train, y_train)
    xgboost_pred += model.predict_proba(test2)[:,1] / 5   #predict probability [0, 1]
    
xgboost_pred = xgboost_pred.round().astype(int)

In [257]:
#get the mean of best fold in catboost  0.8426966292134831  nice  0.77990 selected
                                            # 0.8764044943820225（500）
catboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./catboost_fold{i}.pkl')
    catboost_pred += model.predict(test2) / 5
    
catboost_pred = catboost_pred.round().astype(int)

In [258]:
#get the mean of best fold in logistic  0.8426966292134831 0.78947 selected nice
                                            # 0.8258426966292135(500)
logistic_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./logistic_fold{i}.pkl')
    logistic_pred += model.predict(test2) / 5
    
logistic_pred = logistic_pred.round().astype(int)

In [268]:
# #get the mean of best fold in logistic/randomForest/svc  
# #randomForest win 0.8314606741573034
# log_ran_pred = np.zeros(418)
# for i in range(5):
#     model = joblib.load(f'./log_randomf_fold{i}.pkl')
#     log_ran_pred += model.predict(test2) / 5
    
# log_ran_pred = log_ran_pred.round().astype(int)

In [ ]:
#get the mean of best fold in svc  0.7528089887640449    0.66985
                                        # 0.6910112359550562 (500)
svc_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./svc_fold{i}.pkl')
    svc_pred += model.predict(test2) / 5
    
svc_pred = svc_pred.round().astype(int)

In [260]:
#get the mean of best fold in randomForest   0.8370786516853933/0.8314606741573034(add min)  0.69138/0.78468(min)
                                                # 0.8539325842696629(500)
rf_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./rf_fold{i}.pkl')
    rf_pred += model.predict(test2) / 5

    
rf_pred = rf_pred.round().astype(int)

In [261]:
#get the mean of best fold in decisionTree   0.8258426966292135   0.78947 selected nice  
                                                # 0.8370786516853933(500)
dt_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./dt_fold{i}.pkl')
    dt_pred += model.predict(test2) / 5

    
dt_pred = dt_pred.round().astype(int)

#### way 1:

In [298]:
#combine lgb_pred rf_pred
f_pro = (lgb_pred + xgboost_pred + catboost_pred + logistic_pred)/4
# f_pro = catboost_pred

In [299]:
pred = []
for i in f_pro:
    if i > 0.5:
        pred.append(1)
    else:
        pred.append(0)

#### way 2:

In [300]:
# pred2 = []
# for i in range(418):
#     count_0 = 0
#     count_1 = 0
#     for model in [lgb_pred, xgboost_pred, catboost_pred, logistic_pred]:
#         if model[i] == 0:
#             count_0 += 1
#         else:
#             count_1 += 1
            
#     if count_0 >= count_1:
#         pred2.append(0)
#     else:
#         pred2.append(1)
# # print(pred2)   

In [301]:
sub.Survived = pred

In [302]:
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [303]:
sub.to_csv('submission_new_model.csv', index=False)